Python

In [ ]:
import csv
import uuid #universally unique id
from uuid import UUID

c1 = []
c2 = []
c3 = []
c4 = []
c5 = []
c6 = []
c7 = []
c8 = []
c9 = []
c10 = []
c11 = []
c12 = []
c13 = []
c14 = []
c15 = []

with open('pp-monthly-update-new-version.csv', mode='r') as infile:
    reader = csv.reader(infile, delimiter=',')
   
    for row in reader:
        c1.append(row[0])
        c2.append(row[1])
        c3.append(row[2])
        c4.append(row[3])
        c5.append(row[4])
        c6.append(row[5])
        c7.append(row[6])
        c8.append(row[7])
        c9.append(row[8])
        c10.append(row[9])
        c11.append(row[10])
        c12.append(row[11])
        c13.append(row[12])
        c14.append(row[13])
        c15.append(row[14])


In [ ]:
import pandas as pd

df = pd.DataFrame({"Transaction ID" :c1, "Price":c2, "Date":c3, "Postcode":c4, "N/A1":c5, "N/A2":c6, "N/A3":c7, "Door Number/House Name":c8, "Property Type":c9,
 "Address":c10, "Village/Town":c11, "Town/City":c12, "Location":c13, "N/A4":c14, "N/A5":c15}) #add each list to DataFrame under each named column

#df['Property ID'] = [uuid.uuid4() for _ in range(len(df.index))] #create a uuid for each row in DataFrame and add to new column
df['Property ID'] = df["Door Number/House Name"].replace(' ', '_', regex=True)+"-"+df["Postcode"].replace(' ', '_', regex=True) #create unique property ID using door number/house name and postcode

df["Date"] = pd.to_datetime(df["Date"]).dt.strftime('%d/%m/%Y') #update format to dd/mm/yy (Y is full year format; yy is short year format)

df = df[["Property ID", "Transaction ID", "Price", "Date", "Postcode", "N/A1", "N/A2", "N/A3", "Door Number/House Name", "Property Type",
 "Address", "Village/Town", "Town/City", "Location", "N/A4", "N/A5"]] #rearrange column order in DataFrame

#df.head() #display first 5 rows in DataFrame

In [ ]:
import os 
import json
from json import JSONEncoder
property_data_file = f"property_data"
create_file = os.path.join(property_data_file+".json") #add file type here

try:
    with open(create_file, "w") as fp: #specify path here - create data.json file
        #Dealing with no UUID serialization support in json
        JSONEncoder_olddefault = JSONEncoder.default
        def JSONEncoder_newdefault(self, o):
            if isinstance(o, UUID): return str(o)
            return JSONEncoder_olddefault(self, o)
        JSONEncoder.default = JSONEncoder_newdefault
                    
        json.dump(df.to_dict("dict"), fp,  indent=4) #save DataFrame as dictionary in json file
except RuntimeError:
    print("File not saved")

Python Apache Beam

In [ ]:
import csv
import apache_beam as beam
from apache_beam.transforms import PTransform 
import json

p1 = beam.Pipeline() #initialize pipeline

'''
Flatten a PCollection of lists of strings into a PCollection of strings - use a generator to iterate over the input list and yield each of the elements
Each yielded result in the generator is an element in the resulting PCollection
'''
def generate_elements(elements):
  for element in elements:
    yield element

def parse_file(element):
  for line in csv.reader([element], quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True):
    return line

parsed_csv = (
                p1 
                | 'Read input file' >> beam.io.ReadFromText('pp-monthly-update-new-version.csv')
                | 'Parse file' >> beam.Map(parse_file)
                |'Flatten lists' >> beam.FlatMap(generate_elements)
                #|"split" >> beam.ParDo(lambda row: row.split(" "))
                | "pair" >> beam.Map(lambda w: (w, 1))
                | "group" >> beam.CombinePerKey(sum)
                #| beam.Map(print)
                #|beam.io.WriteToText('test.csv')
                #| 'format json' >> beam.Map(json.dumps)
                #| 'write to text' >> beam.io.WriteToText('output.json')
             )
# lets collect our result with a map transformation into output array
output = []
def collect(row):
    output.append(row)
    return True

parsed_csv | "print" >> beam.Map(collect)
result = p1.run() #run pipeline 

# lets wait until result a available
result.wait_until_finish()

# print the output
print(output)